## 작업순서

1. 데이터 불러오기
2. Nan 값 확인
3. 인코딩
4. 학습, 테스트 데이터 분리
5. 정규화
6. 모델 불러오기, 그리드 서치 적용
7. roc score 확인
8. 예측 값 확인
9. csv 파일로 생성하기 위해 병합

---

### 문제
고객 3,500명에 대한 학습용 데이터(y_train.csv, X_train.csv)를 이용하여 성별예측 모형을 만든 후, 이를 평가용 데이터(X_test.csv)에 적용하여 얻은 2,482명 고객의 성별 예측값(남자일 확률)을 다음과 같은 형식의 CSV 파일로 생성하시오.(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)

### 1.데이터 불러오기

In [1]:
import pandas as pd

In [2]:
X_train = pd.read_csv('data/X_train.csv', encoding='euc-kr')
X_train.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85


In [3]:
X_test = pd.read_csv('data/X_test.csv', encoding='euc-kr')
X_test.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,3500,70900400,22000000,4050000.0,골프,부산본점,13,1.461538,0.789474,26
1,3501,310533100,38558000,48034700.0,농산물,잠실점,90,2.433333,0.369863,3
2,3502,305264140,14825000,30521000.0,가공식품,본 점,101,14.623762,0.083277,3
3,3503,7594080,5225000,NaN,주방용품,부산본점,5,2.000000,0.000000,47
4,3504,1795790,1411200,NaN,수산품,청량리점,3,2.666667,0.125000,8


In [4]:
y = pd.read_csv('data/y_train.csv', encoding='euc-kr')
y.head()

,cust_id,gender
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0


### 2. Nan값 확인

In [5]:
X_train.isnull().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

In [6]:
# Nan 값 대체
X_train = X_train.fillna(0)

### 3. 인코딩

In [7]:
X_train.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,0.0,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,0.0,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,0.0,보석,본 점,2,1.500000,0.000000,85


In [8]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(X_train['주구매상품'])
a_lables = encoder.transform(X_train['주구매상품'])
X_train['주구매상품'] = a_lables

encoder.fit(X_train['주구매지점'])
b_lables = encoder.transform(X_train['주구매지점'])
X_train['주구매지점'] = b_lables

In [9]:
encoder.fit(X_test['주구매상품'])
a_lables = encoder.transform(X_test['주구매상품'])
X_test['주구매상품'] = a_lables

encoder.fit(X_test['주구매지점'])
b_lables = encoder.transform(X_test['주구매지점'])
X_test['주구매지점'] = b_lables

### 4. 학습, 테스트 데이터 분리

In [10]:
X_train.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,5,0,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,21,19,2,1.500000,0.000000,1
2,2,3197000,1639000,0.0,6,1,2,2.000000,0.000000,1
3,3,16077620,4935000,0.0,5,2,18,2.444444,0.318182,16
4,4,29050000,24000000,0.0,15,8,2,1.500000,0.000000,85


In [11]:
x_train = X_train[X_train.columns[1:]]
x_test = X_test[X_test.columns[1:]]
y_train = y.iloc[:,1:]

### 5. 정규화

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_train)
X_scaled_train = scaler.transform(x_train)
X_scaled_test = scaler.transform(x_test)

### 6-1. 모델 불러오기

In [38]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_scaled_train, y_train)
print(model.score(X_scaled_train,y_train))

# 값
pred_train = model.predict(X_scaled_train)
pred_test = model.predict(X_scaled_test)

# 확률
pred_prob_train = model.predict_proba(X_scaled_train)
pred_prob_test = model.predict_proba(X_scaled_test)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9702857142857143


### 6-2. 그리드 서치 사용

In [28]:
from sklearn.model_selection import GridSearchCV

param = {'n_estimators': [i for i in range(0,100,5)],
         'max_depth' : [None, 1,2,3,4,5],
         'min_child_weight':[1,3], 
         'colsample_bytree':[0.5,0.75]}

grid_model = XGBClassifier()
grid_search = GridSearchCV(grid_model, param, cv=5)
grid_search.fit(X_scaled_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estim...
                                     subsample=None, tree_method=None,
                                     validate_parameters=None, verbosity=None),
             iid='deprecated', n_jobs=None,
             param_grid={'colsample_bytree': [0.5, 0.75],
                         'max_

In [29]:
grid_search.best_params_

{'colsample_bytree': 0.5,
 'max_depth': 2,
 'min_child_weight': 3,
 'n_estimators': 20}

In [30]:
print(grid_search.score(X_scaled_train, y_train))

# 값
pred_search_train = grid_search.predict(X_scaled_train)
pred_search_test = grid_search.predict(X_scaled_test)

# 확률
pred_search_prob_train = grid_search.predict_proba(X_scaled_train)
pred_search_prob_test = grid_search.predict_proba(X_scaled_test)

0.674


### 7. roc-score 확인

In [15]:
from sklearn.metrics import roc_auc_score

In [45]:
from sklearn.metrics import roc_auc_score
roc_auc_score = roc_auc_score(y_train, pred_train)
roc_auc_score

0.9656203725352661

In [31]:
roc_auc_score_search = roc_auc_score(y_train,pred_search_train)
roc_auc_score_search

TypeError: 'numpy.float64' object is not callable

In [20]:
pred_search_train

array([0, 0, 1, ..., 0, 0, 0])

### 8.예측값 확인

In [43]:
pred = []
for i in range(len(pred_prob_test)):
    max = pred_search_prob_test[i].max()
    max = round(max,3)
    pred.append(max)

### 9. 데이터프레임 생성


In [44]:
final = pd.DataFrame({'custid' : X_test['cust_id'], 'gender':pred})
final

,custid,gender
0,3500,0.660
1,3501,0.808
2,3502,0.808
3,3503,0.551
4,3504,0.543
...,...,...
2477,5977,0.516
2478,5978,0.513
2479,5979,0.596
2480,5980,0.607
